### Simulator

In [ ]:
#!/usr/bin/env python
# manual

"""
This script allows you to manually control the simulator or Duckiebot
using the keyboard arrows.
"""

import sys
import argparse
import pyglet
from pyglet.window import key
import numpy as np
import gym
import gym_duckietown
from gym_duckietown.envs import DuckietownEnv
from gym_duckietown.wrappers import UndistortWrapper

# from experiments.utils import save_img

parser = argparse.ArgumentParser()
parser.add_argument('--env-name', default=None)
parser.add_argument('--map-name', default='udem1')
parser.add_argument('--distortion', default=False, action='store_true')
parser.add_argument('--draw-curve', action='store_true', help='draw the lane following curve')
parser.add_argument('--draw-bbox', action='store_true', help='draw collision detection bounding boxes')
parser.add_argument('--domain-rand', action='store_true', help='enable domain randomization')
parser.add_argument('--frame-skip', default=1, type=int, help='number of frames to skip')
parser.add_argument('--seed', default=1, type=int, help='seed')
args = parser.parse_args()

if args.env_name and args.env_name.find('Duckietown') != -1:
    env = DuckietownEnv(
        seed = args.seed,
        map_name = args.map_name,
        draw_curve = args.draw_curve,
        draw_bbox = args.draw_bbox,
        domain_rand = args.domain_rand,
        frame_skip = args.frame_skip,
        distortion = args.distortion,
    )
else:
    env = gym.make(args.env_name)

env.reset()
env.render()

@env.unwrapped.window.event
def on_key_press(symbol, modifiers):
    """
    This handler processes keyboard commands that
    control the simulation
    """

    if symbol == key.BACKSPACE or symbol == key.SLASH:
        print('RESET')
        env.reset()
        env.render()
    elif symbol == key.PAGEUP:
        env.unwrapped.cam_angle[0] = 0
    elif symbol == key.ESCAPE:
        env.close()
        sys.exit(0)

    # Take a screenshot
    # UNCOMMENT IF NEEDED - Skimage dependency
    # elif symbol == key.RETURN:
    #     print('saving screenshot')
    #     img = env.render('rgb_array')
    #     save_img('screenshot.png', img)

# Register a keyboard handler
key_handler = key.KeyStateHandler()
env.unwrapped.window.push_handlers(key_handler)

def update(dt):
    """
    This function is called at every frame to handle
    movement/stepping and redrawing
    """

    action = np.array([0.0, 0.0])

    if key_handler[key.UP]:
        action = np.array([0.44, 0.0])
    if key_handler[key.DOWN]:
        action = np.array([-0.44, 0])
    if key_handler[key.LEFT]:
        action = np.array([0.35, +1])
    if key_handler[key.RIGHT]:
        action = np.array([0.35, -1])
    if key_handler[key.SPACE]:
        action = np.array([0, 0])

    # Speed boost
    if key_handler[key.LSHIFT]:
        action *= 1.5

    obs, reward, done, info = env.step(action)
    print('step_count = %s, reward=%.3f' % (env.unwrapped.step_count, reward))

    if key_handler[key.RETURN]:
        from PIL import Image
        im = Image.fromarray(obs)

        im.save('screen.png')

    if done:
        print('done!')
        env.reset()
        env.render()

    env.render()

pyglet.clock.schedule_interval(update, 1.0 / env.unwrapped.frame_rate)

# Enter main event loop
pyglet.app.run()

env.close()

### Postprocess

In [ ]:
# -*- coding: utf-8 -*-

import glob
import sys
import cv2
import os

# Binarization algorithm, with a given original and annotated pair of image
def binarize_a(img_orig, img_ant):
    
    img_hsv = cv2.cvtColor(img_ant,cv2.COLOR_BGR2HSV)
    
    lowerBound = (10, 0, 0); #HSV
    upperBound = (170, 255, 255); #HSV
    
    mask = cv2.inRange(img_hsv, lowerBound, upperBound)
    mask = ~mask
    
    result = mask
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    result = cv2.morphologyEx(result, cv2.MORPH_OPEN, kernel)
    result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel)
    return result

def binarize_b(img_orig, img_ant):
    img_diff = img_orig - img_ant
    
    res_gray = cv2.cvtColor(img_diff,cv2.COLOR_BGR2GRAY)
    res_gray[res_gray > 0] = 255
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    result = cv2.morphologyEx(res_gray, cv2.MORPH_OPEN, kernel)
    result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel)
    return result

# Wrapper function variable; select here the one you want to use
binarize = binarize_b

# Get the list of available recordings
annot_raw_list = glob.glob('recordings\*_annot.avi')
orig_raw_list = glob.glob('recordings\*_orig.avi')

# Check whether original and annotated recordings number match or not
if len(annot_raw_list) != len(orig_raw_list):
    print("Length mismatch! No postprocess performed.")
    sys.exit()

# Iterate and postprocess every recording
for i in range(len(orig_raw_list)):
    # Open recordings...
    cap_orig = cv2.VideoCapture(orig_raw_list[i])
    cap_annot = cv2.VideoCapture(annot_raw_list[i])
    if not cap_orig.isOpened() or not cap_annot.isOpened():
        print("Could not open files! Continuing...")
        continue
    
    # Check whether recordings hold the same number of frames
    if cap_orig.get(cv2.CAP_PROP_FRAME_COUNT) != cap_annot.get(cv2.CAP_PROP_FRAME_COUNT):
        print("Different video length encountered! Continuing...")
        print("DEBUG: orig frames: %i, annot frames: %i" % (cap_orig.get(cv2.CAP_PROP_FRAME_COUNT), cap_annot.get(cv2.CAP_PROP_FRAME_COUNT)))
        continue
    
    # Open VideoWriter Objects
    fourcc=cv2.VideoWriter_fourcc(*'FFV1')
    fps=20
    framesize=(640,480)
    isColor=True
    filename_orig, _ = os.path.splitext(orig_raw_list[i])
    filename_orig = filename_orig + '_pp.avi'
    if os.path.exists(filename_orig):   # If file exists...
        os.remove(filename_orig)    # ...delete it
    vWriter_orig = cv2.VideoWriter(filename_orig, fourcc, fps, framesize, isColor)
    
    isColor=False
    filename_annot, _ = os.path.splitext(annot_raw_list[i])
    filename_annot = filename_annot + '_pp.avi'
    if os.path.exists(filename_annot):  # If file exists...
        os.remove(filename_annot)   # ...delete it
    vWriter_annot = cv2.VideoWriter(filename_annot, fourcc, fps, framesize, isColor)
    
    if not vWriter_orig.isOpened() or not vWriter_annot.isOpened():
        print("Could not open vide writers! Continuing...")
        vWriter_annot.release()
        vWriter_orig.release()
        continue
    
    # Produce output videos
    print("Processing recording nr. {}...".format(i))
    while cap_orig.isOpened() and cap_annot.isOpened(): # Iterate through every frame
        ret_o, frame_o = cap_orig.read()
        ret_a, frame_a = cap_annot.read()
        if not ret_o or not ret_a:
            break
        
        # Postprocess original recording: convert from BGR to RGB
        vWriter_orig.write(cv2.cvtColor(frame_o,cv2.COLOR_BGR2RGB))
        
        # Postprocess annotated frame: binarize it
        annot_binary = binarize(frame_o, frame_a)
        vWriter_annot.write(annot_binary)
    
    print("Processing of recording nr. {} done.".format(i))
    
    # Release writer resources
    vWriter_annot.release()
    vWriter_orig.release()

print("Post-processing finished!")
